Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [127]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### Define Foursquare Credentials and Version

In [128]:
explore_url = 'https://api.foursquare.com/v2/venues/explore?client_id=O3DOYAYXIJZX4KGIGDBXJO3FCQJ5NLIXM2KCFHLF3HAGV3U5&client_secret=HFXRZ0QIJ4A3ODKRGHETHSVNZIDGR0CMOZMT5STUJ5BPJC23&ll= 30.034676, 31.463362&llAcc=10000&alt=0&altAcc=10000&v=20180604&radius=5300&day=any&time=any'
explore_url

'https://api.foursquare.com/v2/venues/explore?client_id=O3DOYAYXIJZX4KGIGDBXJO3FCQJ5NLIXM2KCFHLF3HAGV3U5&client_secret=HFXRZ0QIJ4A3ODKRGHETHSVNZIDGR0CMOZMT5STUJ5BPJC23&ll= 30.034676, 31.463362&llAcc=10000&alt=0&altAcc=10000&v=20180604&radius=5300&day=any&time=any'

In [129]:
explore_results = requests.get(explore_url).json()
explore_results

{'meta': {'code': 200, 'requestId': '5c058e4c9fb6b7351d5c5b6d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-543fc396498e29b4580eaae5-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/dessert_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1d0941735',
         'name': 'Dessert Shop',
         'pluralName': 'Dessert Shops',
         'primary': True,
         'shortName': 'Desserts'}],
       'id': '543fc396498e29b4580eaae5',
       'location': {'address': '279 Meamar Bldg',
        'cc': 'EG',
        'city': 'New Cairo',
        'country': 'مصر',
        'crossStreet': '90 St',
        'distance': 976,
        'formattedAddress': ['279 Meamar Bldg (90 St)',
         'New Cairo, Muḩāfaz̧at al Qāhirah',
         'القاهرة',
         'مصر'],
        'la

In [130]:
items = explore_results['response']['groups'][0]['items']

#### Tranform the data into a *pandas* dataframe

In [53]:
columns_titles =['Venue', 'Category', 'distance_to_center','Latitude', 'Longitude']
newcairo_venues = pd.DataFrame(columns = columns_titles)
for i in items:
    venue_name = i ['venue']['name']
    venue_category = i ['venue']['categories'][0]['shortName']
    distance_to_center = i ['venue']['location']['distance']
    lat = i ['venue']['location']['labeledLatLngs'][0]['lat']
    lng = i ['venue']['location']['labeledLatLngs'][0]['lat']
    newcairo_venues = newcairo_venues.append({'distance_to_center' : distance_to_center, 'Venue' : venue_name,  'Category' :  venue_category, 'Latitude' : lat, 'Longitude' : lng}, ignore_index=True)
newcairo_venues

,Venue,Category,distance_to_center,Latitude,Longitude
0,Coppermelt,Desserts,976,30.026367,30.026367
1,Seoudi (سعودي),Supermarket,1328,30.040625,30.040625
2,Dusit Thani,Resort,1266,30.025706,30.025706
3,The WaterWay,Residential,1391,30.041666,30.041666
4,Ruen Thai,Thai,1238,30.025830,30.025830
5,Andiamo Pizza Garden & Bar,Pizza,1511,30.024557,30.024557
6,Starbucks (ستاربكس),Coffee Shop,1352,30.040931,30.040931
7,Tolip El Narges,Hotel,1597,30.020402,30.020402
8,Doppio Zero,Mediterranean,1227,30.038384,30.038384
9,Sedra,Desserts,1024,30.025774,30.025774


### classifying venues into 10 zones based on its distance from the center 

In [131]:
newcairo_venues['Zones']=newcairo_venues['distance_to_center'].apply(lambda x: 'zone 1'  if x<=500 else ('zone 2' if 500 < x <= 1000 else \
                                                                                                         ('zone 3' if 1000 < x <= 1500 else\
                                                                                                         ('zone 4'if  1500 < x <= 2000 else\
                                                                                                         ('zone 5' if 2000 < x <= 2500 else \
                                                                                                         ('zone 6' if 2500 < x <= 3000 else \
                                                                                                         ('zone 7' if 3000 < x <= 3500 else \
                                                                                                         ('zone 8' if 3500 < x <= 4000 else \
                                                                                                         ('zone 9' if 4000 < x <= 4500 else \
                                                                                                         ('zone 10' if 4500 < x <= 5000 else x
                                                                                                          ))))))))))
newcairo_venues


,Venue,Category,distance_to_center,Latitude,Longitude,Zones
0,Coppermelt,Desserts,976,30.026367,30.026367,zone 2
1,Seoudi (سعودي),Supermarket,1328,30.040625,30.040625,zone 3
2,Dusit Thani,Resort,1266,30.025706,30.025706,zone 3
3,The WaterWay,Residential,1391,30.041666,30.041666,zone 3
4,Ruen Thai,Thai,1238,30.025830,30.025830,zone 3
5,Andiamo Pizza Garden & Bar,Pizza,1511,30.024557,30.024557,zone 4
6,Starbucks (ستاربكس),Coffee Shop,1352,30.040931,30.040931,zone 3
7,Tolip El Narges,Hotel,1597,30.020402,30.020402,zone 4
8,Doppio Zero,Mediterranean,1227,30.038384,30.038384,zone 3
9,Sedra,Desserts,1024,30.025774,30.025774,zone 3


## #. Analyze Each Zone

In [132]:
newcairo_venues.groupby('Zones').count()

,Venue,Category,distance_to_center,Latitude,Longitude
Zones,,,,,
zone 2,4,4,4,4,4
zone 3,15,15,15,15,15
zone 4,5,5,5,5,5
zone 5,4,4,4,4,4
zone 6,1,1,1,1,1
zone 7,1,1,1,1,1


In [133]:
# one hot encoding
newcairo_onehot = pd.get_dummies(newcairo_venues[['Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newcairo_onehot['Zones'] = newcairo_venues['Zones'] 

# move neighborhood column to the first column
fixed_columns = [newcairo_onehot.columns[-1]] + list(newcairo_onehot.columns[:-1])
newcairo_onehot = newcairo_onehot[fixed_columns]

newcairo_onehot.head()

,Zones,American,Arcade,Bar,Café,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai
0,zone 2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,zone 3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,zone 3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,zone 3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,zone 3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [134]:
newcairo_grouped = newcairo_onehot.groupby('Zones').mean().reset_index()
newcairo_grouped

,Zones,American,Arcade,Bar,Café,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai
0,zone 2,0.0,0.25,0.0,0.000000,0.0,0.000000,0.250000,0.25,0.00,0.000000,0.0,0.000000,0.00,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
1,zone 3,0.0,0.00,0.0,0.066667,0.2,0.066667,0.066667,0.00,0.00,0.066667,0.0,0.066667,0.00,0.00,0.066667,0.066667,0.0,0.066667,0.066667,0.00,0.0,0.066667,0.066667,0.066667
2,zone 4,0.2,0.00,0.2,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.2,0.000000,0.00,0.00,0.000000,0.000000,0.2,0.000000,0.000000,0.00,0.2,0.000000,0.000000,0.000000
3,zone 5,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.25,0.250000,0.0,0.000000,0.25,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000
4,zone 6,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,1.000000,0.000000,0.000000
5,zone 7,0.0,0.00,0.0,0.000000,1.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000


### Defining Top 5 venues in each zone

In [135]:
num_top_venues = 5

for zone in newcairo_grouped['Zones']:
    print("----"+zone+"----")
    temp = newcairo_grouped[newcairo_grouped['Zones'] == zone].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----zone 2----
                   venue  freq
0               Desserts  0.25
1  Entertainment Service  0.25
2                 Arcade  0.25
3                 Market  0.25
4               American  0.00


----zone 3----
          venue  freq
0   Coffee Shop  0.20
1          Thai  0.07
2         Sushi  0.07
3          Café  0.07
4  Comfort Food  0.07


----zone 4----
         venue  freq
0     American   0.2
1          Bar   0.2
2  Sports Club   0.2
3        Pizza   0.2
4        Hotel   0.2


----zone 5----
           venue  freq
0           Mall  0.25
1        Seafood  0.25
2      Gift Shop  0.25
3  Gym / Fitness  0.25
4         Market  0.00


----zone 6----
         venue  freq
0  Supermarket   1.0
1     American   0.0
2       Arcade   0.0
3        Sushi   0.0
4  Sports Club   0.0


----zone 7----
         venue  freq
0  Coffee Shop   1.0
1     American   0.0
2       Market   0.0
3        Sushi   0.0
4  Supermarket   0.0




In [136]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [137]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zones']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zones_venues_sorted = pd.DataFrame(columns=columns)
zones_venues_sorted['Zones'] = newcairo_grouped['Zones']

for ind in np.arange(newcairo_grouped.shape[0]):
    zones_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newcairo_grouped.iloc[ind, :], num_top_venues)

zones_venues_sorted

,Zones,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,zone 2,Arcade,Desserts,Entertainment Service,Market,Thai,Hotel,Bar,Café,Coffee Shop,Comfort Food
1,zone 3,Coffee Shop,Thai,Mediterranean,Café,Comfort Food,Desserts,Gym / Fitness,Sushi,Lebanese,Mobile Phones
2,zone 4,American,Hotel,Sports Club,Bar,Pizza,Arcade,Café,Coffee Shop,Comfort Food,Desserts
3,zone 5,Seafood,Gift Shop,Mall,Gym / Fitness,Thai,Hotel,Arcade,Bar,Café,Coffee Shop
4,zone 6,Supermarket,Thai,Hotel,Arcade,Bar,Café,Coffee Shop,Comfort Food,Desserts,Entertainment Service
5,zone 7,Coffee Shop,Thai,Sushi,Arcade,Bar,Café,Comfort Food,Desserts,Entertainment Service,Gift Shop


### Clustering Zones


In [138]:
# set number of clusters
kclusters = 6

newcairo_grouped_clustering = newcairo_grouped.drop('Zones', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newcairo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:6] 


array([3, 5, 4, 1, 0, 2], dtype=int32)

In [139]:
newcairo_grouped_clustering

,American,Arcade,Bar,Café,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai
0,0.0,0.25,0.0,0.000000,0.0,0.000000,0.250000,0.25,0.00,0.000000,0.0,0.000000,0.00,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
1,0.0,0.00,0.0,0.066667,0.2,0.066667,0.066667,0.00,0.00,0.066667,0.0,0.066667,0.00,0.00,0.066667,0.066667,0.0,0.066667,0.066667,0.00,0.0,0.066667,0.066667,0.066667
2,0.2,0.00,0.2,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.2,0.000000,0.00,0.00,0.000000,0.000000,0.2,0.000000,0.000000,0.00,0.2,0.000000,0.000000,0.000000
3,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.25,0.250000,0.0,0.000000,0.25,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000
4,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,1.000000,0.000000,0.000000
5,0.0,0.00,0.0,0.000000,1.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000


In [141]:
newcairo_merged = newcairo_grouped

# add clustering labels
newcairo_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newcairo_merged = newcairo_merged.join(zones_venues_sorted.set_index('Zones'), on='Zones')

newcairo_merged

,Zones,American,Arcade,Bar,Café,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,zone 2,0.0,0.25,0.0,0.000000,0.0,0.000000,0.250000,0.25,0.00,0.000000,0.0,0.000000,0.00,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,3,Arcade,Desserts,Entertainment Service,Market,Thai,Hotel,Bar,Café,Coffee Shop,Comfort Food
1,zone 3,0.0,0.00,0.0,0.066667,0.2,0.066667,0.066667,0.00,0.00,0.066667,0.0,0.066667,0.00,0.00,0.066667,0.066667,0.0,0.066667,0.066667,0.00,0.0,0.066667,0.066667,0.066667,5,Coffee Shop,Thai,Mediterranean,Café,Comfort Food,Desserts,Gym / Fitness,Sushi,Lebanese,Mobile Phones
2,zone 4,0.2,0.00,0.2,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.2,0.000000,0.00,0.00,0.000000,0.000000,0.2,0.000000,0.000000,0.00,0.2,0.000000,0.000000,0.000000,4,American,Hotel,Sports Club,Bar,Pizza,Arcade,Café,Coffee Shop,Comfort Food,Desserts
3,zone 5,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.25,0.250000,0.0,0.000000,0.25,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000,1,Seafood,Gift Shop,Mall,Gym / Fitness,Thai,Hotel,Arcade,Bar,Café,Coffee Shop
4,zone 6,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,1.000000,0.000000,0.000000,0,Supermarket,Thai,Hotel,Arcade,Bar,Café,Coffee Shop,Comfort Food,Desserts,Entertainment Service
5,zone 7,0.0,0.00,0.0,0.000000,1.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,2,Coffee Shop,Thai,Sushi,Arcade,Bar,Café,Comfort Food,Desserts,Entertainment Service,Gift Shop


### Examine Clusters


### Cluster 1

In [142]:
newcairo_merged.loc[newcairo_merged['Cluster Labels'] == 0, newcairo_merged.columns[[1] + list(range(5, newcairo_merged.shape[1]))]]

,American,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,Supermarket,Thai,Hotel,Arcade,Bar,Café,Coffee Shop,Comfort Food,Desserts,Entertainment Service


### Cluster 2

In [143]:
newcairo_merged.loc[newcairo_merged['Cluster Labels'] == 1, newcairo_merged.columns[[1] + list(range(5, newcairo_merged.shape[1]))]]

,American,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,0.0,0.0,0.0,0.0,0.0,0.25,0.25,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,1,Seafood,Gift Shop,Mall,Gym / Fitness,Thai,Hotel,Arcade,Bar,Café,Coffee Shop


### Cluster 3

In [144]:
newcairo_merged.loc[newcairo_merged['Cluster Labels'] == 2, newcairo_merged.columns[[1] + list(range(5, newcairo_merged.shape[1]))]]

,American,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Coffee Shop,Thai,Sushi,Arcade,Bar,Café,Comfort Food,Desserts,Entertainment Service,Gift Shop


### Cluster 4

In [145]:
newcairo_merged.loc[newcairo_merged['Cluster Labels'] == 3, newcairo_merged.columns[[1] + list(range(5, newcairo_merged.shape[1]))]]

,American,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.0,0.0,0.0,0.25,0.25,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Arcade,Desserts,Entertainment Service,Market,Thai,Hotel,Bar,Café,Coffee Shop,Comfort Food


### Cluster 5

In [146]:
newcairo_merged.loc[newcairo_merged['Cluster Labels'] == 4, newcairo_merged.columns[[1] + list(range(5, newcairo_merged.shape[1]))]]

,American,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,4,American,Hotel,Sports Club,Bar,Pizza,Arcade,Café,Coffee Shop,Comfort Food,Desserts


### Cluster 6

In [147]:
newcairo_merged.loc[newcairo_merged['Cluster Labels'] == 5, newcairo_merged.columns[[1] + list(range(5, newcairo_merged.shape[1]))]]

,American,Coffee Shop,Comfort Food,Desserts,Entertainment Service,Gift Shop,Gym / Fitness,Hotel,Lebanese,Mall,Market,Mediterranean,Mobile Phones,Pizza,Residential,Resort,Seafood,Sports Club,Supermarket,Sushi,Thai,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,0.0,0.2,0.066667,0.066667,0.0,0.0,0.066667,0.0,0.066667,0.0,0.0,0.066667,0.066667,0.0,0.066667,0.066667,0.0,0.0,0.066667,0.066667,0.066667,5,Coffee Shop,Thai,Mediterranean,Café,Comfort Food,Desserts,Gym / Fitness,Sushi,Lebanese,Mobile Phones
